In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
import numpy as np
import glob
from PIL import Image, ExifTags

# pip install torchsummary
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from time import time
import pandas as pd
import random

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from matplotlib.patches import Polygon, Rectangle
import pylab
pylab.rcParams['figure.figsize'] = (6,6)

# Own imports 
from config import * 
from utils import *
from data_loader import TacoDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
img_size = 1024

train_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.7),
    A.RandomBrightnessContrast(p=0.2),
    A.RGBShift(10, 10, 10, p=0.3),
    A.GaussNoise(p=0.5),
    A.Normalize(), # If you want to visualize - comment this line 
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc',
                            label_fields=['labels'],
                            min_visibility=0.3, # min visibility of the original area in case of a crop
                           )
)

test_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize(),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc',
                            label_fields=['labels'],
                           )
)

In [5]:
trainset = TacoDataset( 'train', transforms=train_transform, test_size=0.2) # test_transform for no augment
valset   = TacoDataset('val', transforms=test_transform, test_size=0.2)
testset  = TacoDataset('test', transforms=test_transform, test_size=0.2)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)
val_loader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)

In [7]:
next(iter(train_loader))[0]

(tensor([[[ 0.3652,  0.7933,  1.0673,  ...,  0.0398, -0.3198, -0.5596],
          [ 0.0398,  0.0741,  0.2111,  ...,  0.0569, -0.0458, -0.5424],
          [-0.7822, -0.9363, -0.5082,  ...,  0.0569, -0.0458, -0.3198],
          ...,
          [-0.3198, -0.3541, -0.2684,  ..., -0.1828, -0.2342, -0.2856],
          [-0.0458, -0.6794, -0.5596,  ..., -0.0972, -0.1828, -0.2342],
          [ 0.1939, -1.0219, -1.1932,  ..., -0.0287, -0.3712, -0.3883]],
 
         [[ 0.9755,  1.4657,  1.7458,  ...,  0.9755,  0.5203,  0.1702],
          [ 0.5728,  0.7129,  0.8880,  ...,  1.1155,  0.9580,  0.4153],
          [-0.3200, -0.3725,  0.1352,  ...,  1.1856,  1.0980,  0.8004],
          ...,
          [ 0.5028,  0.4678,  0.5553,  ..., -0.2850, -0.3550, -0.4251],
          [ 0.8179,  0.1527,  0.2577,  ..., -0.2500, -0.3550, -0.3901],
          [ 1.0280, -0.3200, -0.5476,  ..., -0.3200, -0.6352, -0.5826]],
 
         [[-0.0615,  0.4265,  0.7054,  ...,  1.3154,  0.8971,  0.5311],
          [-0.2358, -0.1661,

In [8]:
# TODO
# def train(model, train_loader, test_loader, loss_function, optimizer, num_epochs, model_name, lr_scheduler=None, save_model=False ):
    
# #     def loss_fun(output, target):
# #         return F.cross_entropy(output, target)
    
#     out_dict = {'train_acc': [],
#                 'test_acc': [],
#                 'train_loss': [],
#                 'test_loss': []}
  
#     for epoch in tqdm(range(num_epochs), unit='epoch'):
#         model.train()
#         train_correct = 0
#         train_len = 0
#         train_loss = []
#         for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
#             data, target = data.to(device), target.to(torch.float32).to(device)
#             optimizer.zero_grad()
#             output = model(data)[:,0]
#             loss = loss_function(output, target)
#             loss.backward()
#             optimizer.step()
#             if lr_scheduler is not None:
#                 lr_scheduler.step()
#             train_loss.append(loss.item())
#             predicted = output > 0.5
#             train_correct += (target==predicted).sum().cpu().item()
#             train_len += data.shape[0]
            
#         test_loss = []
#         test_correct = 0
#         test_len = 0
#         model.eval()
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(torch.float32).to(device)
#             with torch.no_grad():
#                 output = model(data)[:,0]
#             test_loss.append(loss_function(output, target).cpu().item())
#             predicted = output > 0.5
#             test_correct += (target==predicted).sum().cpu().item()
#             test_len += data.shape[0]

#         if save_model and epoch > 0 and test_correct/test_len > max(out_dict['test_acc']):
#             torch.save(model, 'models/' + model_name)
            
            
#         out_dict['train_acc'].append(train_correct/train_len)
#         out_dict['test_acc'].append(test_correct/test_len)
#         out_dict['train_loss'].append(np.mean(train_loss))
#         out_dict['test_loss'].append(np.mean(test_loss))

        
#         print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
#               f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
        
#     return out_dict

In [9]:
# TODO
# def transfer_model_set(model, freeze_convs=False,):
    
#     if freeze_convs:
#         print('Freezing Convs')
#         # freeze the feature extractors
#         for param in model.parameters():
#             param.requires_grad = False
    
#     if type(model) == models.densenet.DenseNet:
#         in_features = model.classifier.in_features
    
#     elif type(model) == models.resnet.ResNet:
#         in_features = model.fc.in_features
    
    
#     size_hidden = 512
#     out_features = 1
    
#     head = nn.Sequential(
#                     nn.Linear(in_features, size_hidden),
#                     nn.Dropout(DROP_OUT_RATE),
#                     nn.ReLU(),
#                     nn.BatchNorm1d(size_hidden),
#                     nn.Linear(size_hidden, out_features),
#                     nn.Sigmoid()        
#     )
                    
    
#     if type(model) == models.densenet.DenseNet:
#         model.classifier = head
    
#     elif type(model) == models.resnet.ResNet:
#         model.fc = head

#     else:
#         raise Exception('Not implemented the classifier for this type of model')

#     model = model.to(device)

#     return model

In [10]:
# TODO
# HEAD_LEARNING_RATE = 0.001
# NUM_EPOCHS = 5
# loss = nn.BCELoss()
# DROP_OUT_RATE = 0.5

# model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
# model = transfer_model_set(model, freeze_convs=False)
# optimizer = torch.optim.Adam(model.parameters(), HEAD_LEARNING_RATE)

# out_dict = train(model,
#                           train_loader,
#                           test_loader,
#                           loss,
#                           optimizer,
#                           NUM_EPOCHS, 
#                           save_model=True, 
#                           model_name='densenet121_full_Adam')

# # saving results
# d = out_dict
# # with open(f'results/results_transfer_densenet_full_{NUM_EPOCHS}_epochs_{FULL_LEARNING_RATE:.0e}_lr_{optim}_optim.csv', 'w') as csvFile:
# #     writer = csv.writer(csvFile)
# #     writer.writerow(d.keys())
# #     writer.writerows(zip(*d.values()))